# Pembuatan Model machine Learning

## Import Libry yang di gunakan

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


## Load Data

In [ ]:
df = pd.read_csv('../projeck/hasil_pelabelan.csv')

In [3]:
df.head()

,content,score,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,"aplikasi nya bagus, aku sering cek out apa apa...",3,aplikasi nya bagus aku sering cek out apa apa ...,aplikasi nya bagus aku sering cek out apa apa ...,aplikasi nya bagus aku sering cek out apa apa ...,"['aplikasi', 'nya', 'bagus', 'aku', 'sering', ...","['aplikasi', 'bagus', 'cek', 'iklannya', 'kond...",aplikasi bagus cek iklannya kondisikan kdang b...,-14,negative
1,Bagus sering banyak diskon atau harga harga ya...,4,Bagus sering banyak diskon atau harga harga ya...,bagus sering banyak diskon atau harga harga ya...,bagus sering banyak diskon atau harga harga ya...,"['bagus', 'sering', 'banyak', 'diskon', 'atau'...","['bagus', 'diskon', 'harga', 'harga', 'anjlok'...",bagus diskon harga harga anjlok terkadang suka...,-12,negative
2,"Apk yg bagus,,memesan brg & lain sebagiannya j...",5,Apk yg bagusmemesan brg lain sebagiannya jadi...,apk yg bagusmemesan brg lain sebagiannya jadi...,aplikasi yang bagusmemesan barang lain sebagia...,"['aplikasi', 'yang', 'bagusmemesan', 'barang',...","['aplikasi', 'bagusmemesan', 'barang', 'sebagi...",aplikasi bagusmemesan barang sebagiannya mudah...,5,positive
3,"aplikasi aneh, akun SDH gold bahkan platinum, ...",1,aplikasi aneh akun SDH gold bahkan platinum ka...,aplikasi aneh akun sdh gold bahkan platinum ka...,aplikasi aneh akun sudah gold bahkan platinum ...,"['aplikasi', 'aneh', 'akun', 'sudah', 'gold', ...","['aplikasi', 'aneh', 'akun', 'gold', 'platinum...",aplikasi aneh akun gold platinum diskon pas di...,-14,negative
4,"pertama nya bagus belanja via COD lancar, beba...",1,pertama nya bagus belanja via COD lancar bebas...,pertama nya bagus belanja via cod lancar bebas...,pertama nya bagus belanja via bayar ditempat l...,"['pertama', 'nya', 'bagus', 'belanja', 'via', ...","['bagus', 'belanja', 'via', 'bayar', 'ditempat...",bagus belanja via bayar ditempat lancar bebas ...,-2,negative


## Data Slpiting

In [4]:
# Hapus NaN dari X dan y secara bersamaan
df = df.dropna(subset=['text_akhir', 'polarity'])

# Pisahkan kembali fitur dan label
X = df['text_akhir']
y = df['polarity']

In [5]:
# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=5000, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

In [6]:
# Konversi hasil ekstraksi fitur menjadi dataframe
model_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [ ]:
# Bagi data menjadi data latih dan data uji (stratify agar proporsi kelas tetap terjaga)
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
)

# Simpan distribusi awal data latih (sebelum sampling)
y_train_original = y_train.copy()

# Oversampling (menambah kelas minoritas)
oversampler = RandomOverSampler(random_state=42)
X_train_over, y_train_over = oversampler.fit_resample(X_train, y_train)

# Undersampling (mengurangi kelas mayoritas)
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

# Pilih metode yang dipakai untuk training: 'oversampling' atau 'undersampling'
sampling_method = 'oversampling'

if sampling_method == 'oversampling':
    X_train_final, y_train_final = X_train_over, y_train_over
elif sampling_method == 'undersampling':
    X_train_final, y_train_final = X_train_under, y_train_under
else:
    raise ValueError("sampling_method harus 'oversampling' atau 'undersampling'")

# Gunakan variabel ini agar cell training di bawah tetap kompatibel
X_train, y_train = X_train_final, y_train_final

In [ ]:
# Cek distribusi kelas sebelum dan sesudah sampling
print('Distribusi kelas data asli:')
print(y.value_counts())

print('\nDistribusi kelas data latih awal (sebelum sampling):')
print(pd.Series(y_train_original).value_counts())

print('\nDistribusi kelas setelah OVERSAMPLING:')
print(pd.Series(y_train_over).value_counts())

print('\nDistribusi kelas setelah UNDERSAMPLING:')
print(pd.Series(y_train_under).value_counts())

print(f"\nMetode yang dipakai untuk training: {sampling_method}")

In [9]:
model_df.head(2)

,abaikan,abang,aceh,adaa,adem,adik,adil,adm,admin,administrasi,...,wp,xl,xpress,xtra,yaaaa,yaampun,yahh,yapadahal,yt,zaman
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.224106,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


### Membuat Model Machine Learning

#### Membuat model dengan algoritma Naive Bayes

In [10]:
# Membuat objek model Naive Bayes (Bernoulli Naive Bayes)
naive_bayes = BernoulliNB()

# Melatih model Naive Bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

# Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

# Menampilkan akurasi
print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)

Naive Bayes - accuracy_train: 0.7315016896648832
Naive Bayes - accuracy_test: 0.7178963672205013


In [11]:
# Membuat objek model Logistic Regression
logistic_regression = LogisticRegression()

# Melatih model Logistic Regression pada data pelatihan
logistic_regression.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_lr = logistic_regression.predict(X_train.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# Evaluasi akurasi model Logistic Regression pada data pelatihan
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)

# Evaluasi akurasi model Logistic Regression pada data uji
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

# Menampilkan akurasi
print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

Logistic Regression - accuracy_train: 0.9233490566037735
Logistic Regression - accuracy_test: 0.9124190368910167


In [12]:
# Membuat objek model Decision Tree
decision_tree = DecisionTreeClassifier()

# Melatih model Decision Tree pada data pelatihan
decision_tree.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_dt = decision_tree.predict(X_train.toarray())
y_pred_test_dt = decision_tree.predict(X_test.toarray())

# Evaluasi akurasi model Decision Tree
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

# Menampilkan akurasi
print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)

Decision Tree - accuracy_train: 0.9999119966206702
Decision Tree - accuracy_test: 0.7197268375105604


In [13]:
# Membuat DataFrame untuk hasil akurasi
results_df = pd.DataFrame({
    'Model': ['Naive Bayes', 'Logistic Regression', 'Decision Tree'],
    'Accuracy Train': [accuracy_train_nb, accuracy_train_lr, accuracy_train_dt],
    'Accuracy Test': [accuracy_test_nb, accuracy_test_lr, accuracy_test_dt]
})
# Menampilkan hanya kolom "Accuracy Test"
accuracy_test_only = results_df[['Model', 'Accuracy Test']]
print(accuracy_test_only)

                 Model  Accuracy Test
0          Naive Bayes       0.717896
1  Logistic Regression       0.912419
2        Decision Tree       0.719727


In [14]:
# Mengurutkan DataFrame berdasarkan kolom "Accuracy Test" dari tertinggi ke terendah
accuracy_test_sorted = accuracy_test_only.sort_values(by='Accuracy Test', ascending=False)

# Menampilkan DataFrame yang telah diurutkan
print(accuracy_test_sorted)

                 Model  Accuracy Test
1  Logistic Regression       0.912419
2        Decision Tree       0.719727
0          Naive Bayes       0.717896


In [18]:
df.head(1)

,content,score,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,"aplikasi nya bagus, aku sering cek out apa apa...",3,aplikasi nya bagus aku sering cek out apa apa ...,aplikasi nya bagus aku sering cek out apa apa ...,aplikasi nya bagus aku sering cek out apa apa ...,"['aplikasi', 'nya', 'bagus', 'aku', 'sering', ...","['aplikasi', 'bagus', 'cek', 'iklannya', 'kond...",aplikasi bagus cek iklannya kondisikan kdang b...,-14,negative


In [19]:
# Define your preprocessing functions
def cleaningText(text):
    # Add your text cleaning steps here
    cleaned_text = text.lower()
    # Add more cleaning steps as needed
    return cleaned_text

def casefoldingText(text):
    # Add your casefolding steps here
    casefolded_text = text.lower()
    return casefolded_text

def fix_slangwords(text):
    # Add your slang words fixing steps here
    slang_fixed_text = text
    return slang_fixed_text

def tokenizingText(text):
    # Add your tokenizing steps here
    tokenized_text = text.split()
    return tokenized_text

def filteringText(text):
    # Add your filtering steps here
    filtered_text = [word for word in text if word.isalpha()]
    return filtered_text

def toSentence(list_words):
    # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

# Input kalimat baru dari pengguna
kalimat_baru = input("Masukkan kalimat baru: ")

# Melakukan preprocessing pada kalimat baru
kalimat_baru_cleaned = cleaningText(kalimat_baru)
kalimat_baru_casefolded = casefoldingText(kalimat_baru_cleaned)
kalimat_baru_slangfixed = fix_slangwords(kalimat_baru_casefolded)
kalimat_baru_tokenized = tokenizingText(kalimat_baru_slangfixed)
kalimat_baru_filtered = filteringText(kalimat_baru_tokenized)
kalimat_baru_final = toSentence(kalimat_baru_filtered)

# Menggunakan objek tfidf yang sudah di-fit dari pelatihan sebelumnya
X_kalimat_baru = tfidf.transform([kalimat_baru_final])

# Memperoleh prediksi sentimen kalimat baru
prediksi_sentimen = logistic_regression.predict(X_kalimat_baru)
# Menampilkan hasil prediksi
# Menampilkan hasil prediksi
if prediksi_sentimen[0] == 'positive':
    print("Sentimen kalimat baru adalah POSITIF.")
else:
    print("Sentimen kalimat baru adalah NEGATIF.")

Masukkan kalimat baru: suka banget sama aplikasinya
Sentimen kalimat baru adalah POSITIF.
